In [1]:
import pandas as pd
import openai
import re
import numpy as np

api_key = ""
openai.api_key = api_key


def askChatGPT(messages):
    MODEL = "gpt-4"
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=messages,
        temperature=1)
    return response['choices'][0]['message']['content'] # type: ignore

In [2]:
# GPT的一些参数的初始化
# role :user assistant system
init = [{
    "role": "system",
    "content": "你需要和我进行一段对话，对话长度和内容都不确定，只有当我说对话结束的时候才意味着我们的谈话结束了，然后你需要给出我的人格特质。当我说谈话开始的时候，意味着我们开启了新一轮的谈话，需要你不考虑之前的所有对话，在对话的过程中你需要扮演对话者，并与我进行对话，对话的过程中不能提到任何与否认你是一个模型的事情。"
}]# 日志
log = []

In [3]:
msg_data = pd.read_csv("../data/CPED/train_split.csv")
msg_data = msg_data.iloc[0:2861]
msg_data.head()

,TV_ID,Dialogue_ID,Utterance_ID,Speaker,Gender,Age,Neuroticism,Extraversion,Openness,Agreeableness,Conscientiousness,Scene,FacePosition_LU,FacePosition_RD,Sentiment,Emotion,DA,Utterance
0,1,01_000,01_000_000,童文洁,female,middle-aged,high,high,low,low,high,other-venue,108_136,156_202,neutral,neutral,greeting,真巧
1,1,01_000,01_000_001,童文洁,female,middle-aged,high,high,low,low,high,other-venue,193_144,253_197,neutral,neutral,greeting,车没事了
2,1,01_000,01_000_002,刘静,female,middle-aged,low,high,high,high,high,other-venue,0_0,0_0,neutral,neutral,answer,是你呀 没事了没事 谢谢你
3,1,01_000,01_000_003,童文洁,female,middle-aged,high,high,low,low,high,other-venue,0_0,0_0,neutral,neutral,question,没事没事 你也去春风
4,1,01_001,01_001_000,刘静,female,middle-aged,low,high,high,high,high,other-venue,0_0,0_0,neutral,neutral,answer,对


# Train

In [27]:
def asd(row):
    for i in range(len(has_record)):
        if i == len(res_a): res_a.append([{"role": "user", "content": "现在聊天开始"}])
        res_a[i].append({
            "role": "user" if has_record[i] == row["Speaker"] else "assistant",
            "content": row["Utterance"]
        })
    return 0

res = [] # 结果保存
for i in np.unique(msg_data["Dialogue_ID"]):
    tmp_a = msg_data[msg_data["Dialogue_ID"] == i] # 获取当前的对话ID所对应样本
    has_record = np.unique(tmp_a["Speaker"])
    res_a = []
    tmp_a.apply(asd, axis=1)
    for i, v in enumerate(res_a):
        v.append({
            "role": "user",
            "content": "现在聊天结束，评估一下我的人格"
        })
        v.append({
            "role": "assistant",
            "content": "你的人格可能是：Neuroticism {}, Extraversion {}, Openness {}, Agreeableness {}, Conscientiousness {}".format(
                np.unique(tmp_a[tmp_a["Speaker"] == has_record[i]]["Neuroticism"])[0],
                np.unique(tmp_a[tmp_a["Speaker"] == has_record[i]]["Extraversion"])[0],
                np.unique(tmp_a[tmp_a["Speaker"] == has_record[i]]["Openness"])[0],
                np.unique(tmp_a[tmp_a["Speaker"] == has_record[i]]["Agreeableness"])[0],
                np.unique(tmp_a[tmp_a["Speaker"] == has_record[i]]["Conscientiousness"])[0]
            )
        })
        res.append(v)

In [28]:
msg_res = []
for i in range(2):
    for j in res[i]:
        msg_res.append(j)
        
msg_res

[{'role': 'user', 'content': '现在聊天开始'},
 {'role': 'assistant', 'content': '真巧'},
 {'role': 'assistant', 'content': '车没事了'},
 {'role': 'user', 'content': '是你呀 没事了没事 谢谢你'},
 {'role': 'assistant', 'content': '没事没事 你也去春风'},
 {'role': 'user', 'content': '现在聊天结束，评估一下我的人格'},
 {'role': 'assistant',
  'content': '你的人格可能是：Neuroticism low, Extraversion high, Openness high, Agreeableness high, Conscientiousness high'},
 {'role': 'user', 'content': '现在聊天开始'},
 {'role': 'user', 'content': '真巧'},
 {'role': 'user', 'content': '车没事了'},
 {'role': 'assistant', 'content': '是你呀 没事了没事 谢谢你'},
 {'role': 'user', 'content': '没事没事 你也去春风'},
 {'role': 'user', 'content': '现在聊天结束，评估一下我的人格'},
 {'role': 'assistant',
  'content': '你的人格可能是：Neuroticism high, Extraversion high, Openness low, Agreeableness low, Conscientiousness high'}]

# Valid

In [30]:
valid_res = [{
    "role": "user",
    "content": "好厨艺倒是真的 不过于春晓"
}, {
    "role": "user",
    "content": "这么恨嫁可不太好吧"
}, {
    "role": "user",
    "content": "现在聊天结束，评估一下我的人格"
}]

In [31]:
ask_msg = [v for v in msg_res]
for i in valid_res:
    ask_msg.append(i)
    
ask_msg

[{'role': 'user', 'content': '现在聊天开始'},
 {'role': 'assistant', 'content': '真巧'},
 {'role': 'assistant', 'content': '车没事了'},
 {'role': 'user', 'content': '是你呀 没事了没事 谢谢你'},
 {'role': 'assistant', 'content': '没事没事 你也去春风'},
 {'role': 'user', 'content': '现在聊天结束，评估一下我的人格'},
 {'role': 'assistant',
  'content': '你的人格可能是：Neuroticism low, Extraversion high, Openness high, Agreeableness high, Conscientiousness high'},
 {'role': 'user', 'content': '现在聊天开始'},
 {'role': 'user', 'content': '真巧'},
 {'role': 'user', 'content': '车没事了'},
 {'role': 'assistant', 'content': '是你呀 没事了没事 谢谢你'},
 {'role': 'user', 'content': '没事没事 你也去春风'},
 {'role': 'user', 'content': '现在聊天结束，评估一下我的人格'},
 {'role': 'assistant',
  'content': '你的人格可能是：Neuroticism high, Extraversion high, Openness low, Agreeableness low, Conscientiousness high'},
 {'role': 'user', 'content': '好厨艺倒是真的 不过于春晓'},
 {'role': 'user', 'content': '这么恨嫁可不太好吧'},
 {'role': 'user', 'content': '现在聊天结束，评估一下我的人格'}]

In [36]:
pre = askChatGPT(ask_msg)

In [33]:
pre

'你的人格可能是：Neuroticism low, Extraversion high, Openness high, Agreeableness high, Conscientiousness high'

In [35]:
pre

'你的人格可能是：Neuroticism low, Extraversion high, Openness high, Agreeableness high, Conscientiousness high'

In [37]:
pre

'你的人格可能是：Neuroticism low, Extraversion high, Openness high, Agreeableness high, Conscientiousness high'